In [1]:
pip install pyspark==3.0.0

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os


spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("Test it 2") \
    .getOrCreate()

sc = spark.sparkContext

In [3]:
sc.stop()

In [1]:
from pyspark.sql import SparkSession

working_directory = 'jars/*'

spark = SparkSession.\
        builder.\
        appName("pyspark-with-mongo").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1g").\
        config("spark.mongodb.input.uri","mongodb://mongodb:27017/datascience.movies").\
        config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").\
        config('spark.driver.extraClassPath', working_directory). \
        getOrCreate()

sc = spark.sparkContext

In [27]:
from pymongo import MongoClient
from pprint import pprint as pp
client = MongoClient('mongodb://mongodb:27017')
db = client.ecommerce

In [29]:
res = db.inventory.insert_one(
    {
        "Item": "Ruler",
        "Colors": ["Red","Green","Blue","Clear","Yellow"],
        "Inventory": {
            "OnHand": 47,
            "MinOnHand": 40
        },
        "UnitPrice": 0.89
    }
)

In [30]:
df = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
    .option("uri", "mongodb://mongodb:27017/ecommerce.inventory")\
    .load()

Py4JJavaError: An error occurred while calling o149.load.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.GatewayConnection.run(GatewayConnection.java:238)
java.lang.Thread.run(Thread.java:748)

The 

In [7]:
df.select('*').show()

+--------------------+---------+-----+---------+--------------------+
|              Colors|Inventory| Item|UnitPrice|                 _id|
+--------------------+---------+-----+---------+--------------------+
|[Red, Green, Blue...| [47, 40]|Ruler|     0.89|[5fd4286194b70759...|
+--------------------+---------+-----+---------+--------------------+



In [8]:
df.registerTempTable("inventory")

In [9]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [10]:
items = sqlContext.sql('''
    select `_id`, `inventory`, `UnitPrice`, `Item` from inventory
''')

In [11]:
items.take(1)

[Row(_id=Row(oid='5fd4286194b70759be8fbbb2'), inventory=Row(OnHand=47, MinOnHand=40), UnitPrice=0.89, Item='Ruler')]

In [12]:
items.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- inventory: struct (nullable = true)
 |    |-- OnHand: integer (nullable = true)
 |    |-- MinOnHand: integer (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- Item: string (nullable = true)



https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions

In [20]:
from pyspark.sql.functions import *

items.select('_id', 'inventory', column('UnitPrice').alias('price')).show()

+--------------------+---------+-----+
|                 _id|inventory|price|
+--------------------+---------+-----+
|[5fd4286194b70759...| [47, 40]| 0.89|
+--------------------+---------+-----+



In [21]:
items.select(current_date()).show()

+--------------+
|current_date()|
+--------------+
|    2020-12-12|
+--------------+



In [23]:
items.select("_id", "Item", "inventory.OnHand").show()

+--------------------+-----+------+
|                 _id| Item|OnHand|
+--------------------+-----+------+
|[5fd4286194b70759...|Ruler|    47|
+--------------------+-----+------+



In [24]:
sc.stop()